# Zalora Scraping

### Kadek Dwi Budi Utama
### https://github.com/kadekutama

2016

### Inisialisasi URL dan Mendapatkan Akses
Kita menggunakan library urllib3 untuk melakukan request dan mengakses Zalora.

Ada 2 macam URL yang kita gunakan, yaitu URL original (contoh: https://www.zalora.co.id/women/tas/?from=header) dan URL yang sudah ditambahkan dengan JSON request. Kita tidak bisa melakukan scraping hanya dengan URL original. Hal ini disebabkan karena Zalora menerapkan JSON pada halamannya, sehingga konten bersifat dinamis (konten baru dimuat apabila jendela browser mencakup area objek). Website dengan konten JSON tidak dapat dilakukan scrape hanya dengan mengandalkan HTML-nya karena script Python yang kita buat nantinya akan menjadi "bot" yang tidak bisa mengakses konten dinamis.

Untuk mengetahui URL JSON request, kita perlu melakukan sniffing pada halaman yang akan kita scrap dengan menggunakan element inspector. Di browser Firefox, kita cukup klik kanan, inspect element, pilih network, terakhir tandai "JS" dan "XHR". JSON request tidak serta merta akan muncul begitu saja di element inspector. Kita harus memancing web server dengan cara klik pada sembarang nomor halaman. Setelah itu, JSON request akan muncul dengan method "GET" dan file "rpc?&req=". Kemudian kita klik pada JSON request tersebut lalu copy request URL pada tab headers. Request URL inilah yang nantinya akan kita gunakan, bukan URL originalnya.

Pada bagian bawah tab headers, kita akan menjumpai request header. Kita gunakan header tersebut dalam mengakses Zalora. Tujuannya adalah agar IP kita tidak diblokir dengan cepat karena web server mengira kita mengaksesnya seolah-olah menggunakan web browser.

In [135]:
import urllib3 as u3

#Link original: https://www.zalora.co.id/women/tas/?from=header
url = "https://www.zalora.co.id/_c/rpc?&req=%7B%22method%22%3A%22Costa.ListCatalogProducts%22%2C%22params%22%3A%5B%7B%22category_id%22%3A%5B%22264%22%5D%2C%22limit%22%3A72%2C%22offset%22%3A0%2C%22segment%22%3A%22women%22%2C%22dir%22%3A%22desc%22%2C%22sort%22%3A%22popularity%22%2C%22catalog_type%22%3A%22%22%2C%22url_key%22%3A%22%2Fwomen%2Ftas%22%7D%5D%7D&lang=id"
header = {
    "Host": "www.zalora.co.id",
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:49.0) Gecko/20100101 Firefox/49.0',
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "X-NewRelic-ID": "VQQEU1dSABACVFhVAAEFUw==",
    "X-Requested-With": "XMLHttpRequest",
    "Referer": "https://www.zalora.co.id/women/tas/?sort=popularity&dir=desc&page=2&category_id=264",
    "Cookie": "zid=953961016.1472871422; 905125419dc9d3c8f422452a786673ba=-4; zalora_referer=a%3A1%3A%7Bs%3A14%3A%22www.google.com%22%3Ba%3A1%3A%7Bi%3A0%3Bs%3A23%3A%22https%3A%2F%2Fwww.google.com%2F%22%3B%7D%7D; nl_slider=collapsed; sl_device_id=252885606634373120; sl-auth-token=4e490f91-e98d-425b-becc-9960bd3a621e; userGender=women; userLanguage=id; browserDetection=eyJ0eXBlIjoiYnJvd3NlciIsIm5hbWUiOiJGaXJlZm94IiwiY3NzQ2xhc3MiOiJmaXJlZm94IiwidmVyc2lvbiI6IjQ5In0%3D; PHPSESSID_83d0767d2f534a23f21399428e28c076=d5306109d6b30a893449b3b5529ac4b4; AWSELB=13F321451EA630C5477012B02AFF87475CB63AC691C77A58C4ECEAAF604BFF99142F3C866A11084484BB2EA98319ADC5A365A0C8D1DAD3454A62D1BF77E2A389A5501844D5; utag_main=v_id:0157b81c9aba001a3720fbc82f580204e003000d0086e$_sn:4$_ss:0$_st:1476279758114$ses_id:1476277638509%3Bexp-session$_pn:12%3Bexp-session; sessionCount=4; vs.bid=fzMOf67Ys2htKd7Xgx4uicM.iu6ot59j; pageCount=12",
    "Connection": "keep-alive"
}

u3.disable_warnings()
http = u3.PoolManager(10, headers=header)
req = http.urlopen('GET', url)
print(req)
page = req.data
print(page[1:1000])

b'"id":null,"result":{"response":{"numFound":6197,"start":0,"minPrice":45000,"maxPrice":5.9e+06,"docs":[{"meta":{"sku":"VE633AC69MUGID","max_saving_percentage":"10","brand":"Verzoni","name":"Nura","special_price":"476.100","price":"529.000","activated_at":"2016-06-09 15:53:14","max_price":"529.000","grouped_products":"VE633AC70MUFID|VE633AC69MUGID|VE633AC68MUHID","sizesystembrand":"International","sizesystembrand_position":"0","id_catalog_config":"1186230","sub_cat_type_id":"304","gender_id":"2","attribute_set_id":"10","categories":"264|510|1921|6483|6484|6494","max_special_price":"476.100","color_family":"brown"},"link":"verzoni-nura-brown-1186230.html","image":"https://dynamic.zacdn.com/5LY8ZhNG_I4b0IS5jM2-zWQ6ZPY=/fit-in/236x345/filters:quality(90):fill(ffffff)/http://static.id.zalora.net/p/verzoni-2000-0326811-1.jpg","image_gallery":"https://dynamic.zacdn.com/24uWdQUEXFIFRydPldUhBJMfzgw=/fit-in/44x44/filters:quality(90):fill(ffffff)/http://static.id.zalora.net/p/verzoni-2000-032681

### Mendapatkan File JSON dan Memanipulasinya
Request yang telah kita lakukan sebelumnya hanya berisi file HTML dasar. Kita tidak dapat melakukan scraping secara langsung. Oleh karena itu, kita perlu membuka akses terhadap file JSON.

Jika kita perhatikan tab response pada element inspector, kita mendapati bahwa informasi mengenai produk tas tersimpan di: result --> response --> docs --> (nomor indeks). Ada pula informasi yang tesimpan di tempat yang lebih dalam, yaitu pada tag "meta". Apabila kita ingin mendapatkan nama produk pada indeks ke-0, maka kita dapat menuliskan perintah: print(prods['result']['response']['docs']['0']['meta']['name']).

In [136]:
import json

id = []
brand = []
nama = []
harga = []
warna = []
image = []

#prods['result']['response']['docs'][0]['meta']['name']
prods = json.loads(page.decode("utf-8"))
for x in prods['result']['response']['docs']:
    id.append(x['meta']['sku'])
    brand.append(x['meta']['brand'])
    nama.append(x['meta']['name'])
    harga.append(int(x['meta']['price'].replace(".","")))
    warna.append(x['meta']['color_family'])
    image.append(x['image'])

### Membuat Data Frame dari List
List yang sudah ada sebelumnya digabung menjadi sebuah "dictionary". Dictionary ini kemudian dikonversi menjadi sebuah tabel.

In [137]:
import pandas as pd

zalora = {
    "ID" : id,
    "Brand" : brand,
    "NamaProduk" : nama,
    "Harga" : harga,
    "Warna" : warna,
    "Gambar" : image
}

df = pd.DataFrame(zalora, columns = ["ID","Brand","NamaProduk","Harga","Warna","Gambar"])
df.sort_values("Harga", ascending=True)

ID               Brand  \
73  AIZQCAC0000073GS              ZALORA   
58  AIZQRAC0000058GS              ZALORA   
56  AAGDBAC0000064GS              ZALORA   
39    HU072AC47AKAID                HUER   
38    PA172AC55NKQID            PALOMINO   
15  ARRCPAC0000048GS              ZALORA   
42  ARQZDAC0000038GS              ZALORA   
49  ARQWFAC0000014GS  Something Borrowed   
11    HU072AC55AJSID                HUER   
50  ARQZMAC0000029GS              ZALORA   
55  AONZPAC0000082GS  Something Borrowed   
71  AONZJAC0000088GS  Something Borrowed   
72    KA772AC24WWPID         Karen&Chloe   
74  ARQZAAC0000041GS              ZALORA   
75  AAGDTAC0000046GS              ZALORA   
76  ARQWIAC0000011GS  Something Borrowed   
25    JA042AC96OMHID            JanSport   
61  AAGBCAC0000015GS              ZALORA   
43    EN997AC63MKMID   EN-JI by Palomino   
40    BI248AC81DJQID           Billabong   
77  ARRAEAC0000011GS              ZALORA   
16    BI248AC83DJOID           Billabong   
9     BI248AC82DJPID           Billabong   
8     BI248AC84DJNID           Billabong   
13    MA533AC55OTSID               Mango   
18    HU072AC19UWUID                HUER   
47    EN997AC75SJGID   EN-JI by Palomino   
7     EN997AC01EUGID   EN-JI by Palomino   
63    EN997AC98EUJID   EN-JI by Palomino   
12    MA533AC44GPRID               Mango   
..               ...                 ...   
65    LE281AC75COAID          Les Catino   
27    AD349AC61TZYID              adidas   
0     VE633AC69MUGID             Verzoni   
14    PA172AC23SDOID            PALOMINO   
41    AD349AC28SBJID              adidas   
2     CA772AC81QOWID            calliope   
53    MA533AC58OTPID               Mango   
62    AD349AC26SBLID              adidas   
6     HU072AC56QTNID                HUER   
17    HU072AC18IGBID                HUER   
69    MA533AC49OTYID               Mango   
48    AD349AC27SBKID              adidas   
57    HU072AC29PYKID                HUER   
23    MA533AC77FECID               Mango   
30    CA929AC37QQSID      Call It Spring   
51    GO748AC87SQMID            Gobelini   
32    SU137AC43DEMID            Superdry   
36    SU137AC29DFAID            Superdry   
22    MA533AC86FDTID               Mango   
19    JA042AC70UAZID            JanSport   
26    PH269AC26NXHID    Phillipe Jourdan   
3     NI126AC50KGRID                Nike   
1     NI126AC51KGQID                Nike   
34    SU137AC41DEOID            Superdry   
33    SU137AC42DENID            Superdry   
70    PH269AC50CZHID    Phillipe Jourdan   
10    GO748AC64XILID            Gobelini   
52    GO748AC71WJCID            Gobelini   
5     PH269AC54CZDID    Phillipe Jourdan   
31    GO748AC72WJBID            Gobelini   

                                     NamaProduk    Harga       Warna  \
73     Raya - Metalcraft Flaphexagonal Smallbag   249000        pink   
58  Raya - Metallicbrocade Round Bagwith Tassel   249000       black   
56                         Mini Zipperedluggage   249000       black   
39              Janira Handbag With Chain Strap   259000       black   
38                                        Arlyn   299000        grey   
15                Looped Strap Medium Sling Bag   299000       white   
42                Structured Classic Saddle Bag   299000        navy   
49                   Chain Handle Crossbody Bag   299000  black|pink   
11                         Mity Classic Handbag   299000        grey   
50    Tri-Toned Chain Handles Angular Sling Bag   299000       green   
55                      Chained Detail Flap Bag   299000       black   
71                  Halfmoon Turnlock Crossbody   299000       black   
72                             Tas Wanita Hitam   299000       black   
74                Structured Classic Saddle Bag   299000       black   
75                          Medium Zipperedhobo   299000       black   
76                   Chain Handle Crossbody Bag   299000  black|navy   
25                                    Half Pint   299000        b

### Konversi Data Frame ke File CSV
Langkah terakhir ialah mengonversi data frame yang telah dibuat menjadi file CSV agar dapat digunakan untuk berbagai keperluan.

In [138]:
df.to_csv("Zalora Scraping.csv",sep=";")